## STEP 01. Import Modules

In [1]:
import numpy as np
import pandas as pd
import json
from tqdm import *

import numpy as np
import pandas as pd

# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt

/usr/local/lib/python3.8/dist-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


In [2]:
import os
os.getcwd();

## STEP 02. Set Target csv File List

In [3]:
list = [ 'data']

## STEP 04. Drawing LDA

In [4]:
df = pd.read_csv("data.csv")
df

,title,cleanBody,category
0,"""아시안 증오를 멈춰라""…미 전역서 규탄·추모 시위","[앵커] 애틀랜타 총격 사건으로 미국 시민들은 또한번 거리로 나왔습니다. 슬프고, ...",international
1,"한인 4명 신원 공개…희생자 아들 ""명백한 증오범죄""",[앵커] 이번 사건에서 희생된 한인 4명의 신원이 사흘 만에 공개됐습니다. 유가족과...,international
2,오늘(12일) 소상공인 버팀목자금 '짝수' 신청날…13일부터 모두 신청,12일 오전 8시부터 '버팀목자금.kr'을 통해 짝수 소상공인들로부터 버팀목자금 신...,economy
3,"""노래방에서 혼자 노래 중입니다""…새벽에도 들어온 '버팀목자금'",11일 오후 서울 황학동 중앙시장 중고가전제품 판매점에 진열된 TV에서 문재인 대통...,economy
4,'실종 2주째' 고양시 발달장애인 점퍼 한강서 발견,'실종 2주째' 고양시 발달장애인 점퍼 한강서 발견 경기 고양시 행주산성둘레길에서 ...,society
...,...,...,...
3133,"日코로나 신규확진 600명…""인슐린주사기로 백신1병에 7회""",일본 수도권 일부 지역에 신종 코로나바이러스 감염증(코로나19) 긴급사태가 발령 중...,society
3134,[슬라이드 뉴스] 이지은 누구? '금홍아 금홍아'로 신인상 휩쓴 '청춘스타',숨진 채 발견된 배우 이지은에 대해 관심이 쏠리고 있다. 1971년생인 이지은은 1...,entertain
3135,"文 “검찰 공정성 신뢰 나아지지 않아… 기소·수사권 분리, 나아가야 할 방향”","문재인 캐리커처 문재인(얼굴) 대통령은 8일 ""기소권과 수사권 분리는 앞으로도 꾸준...",politics
3136,"中, '백신여권' 도입에 ""편리한 인원왕래 요구 절실""",[서울=뉴시스] 중국 외교부가 '중국판 백신 여권'에 대해 추가적인 설명을 내놓았다...,international


In [5]:
df.dropna()

,title,cleanBody,category
0,"""아시안 증오를 멈춰라""…미 전역서 규탄·추모 시위","[앵커] 애틀랜타 총격 사건으로 미국 시민들은 또한번 거리로 나왔습니다. 슬프고, ...",international
1,"한인 4명 신원 공개…희생자 아들 ""명백한 증오범죄""",[앵커] 이번 사건에서 희생된 한인 4명의 신원이 사흘 만에 공개됐습니다. 유가족과...,international
2,오늘(12일) 소상공인 버팀목자금 '짝수' 신청날…13일부터 모두 신청,12일 오전 8시부터 '버팀목자금.kr'을 통해 짝수 소상공인들로부터 버팀목자금 신...,economy
3,"""노래방에서 혼자 노래 중입니다""…새벽에도 들어온 '버팀목자금'",11일 오후 서울 황학동 중앙시장 중고가전제품 판매점에 진열된 TV에서 문재인 대통...,economy
4,'실종 2주째' 고양시 발달장애인 점퍼 한강서 발견,'실종 2주째' 고양시 발달장애인 점퍼 한강서 발견 경기 고양시 행주산성둘레길에서 ...,society
...,...,...,...
3133,"日코로나 신규확진 600명…""인슐린주사기로 백신1병에 7회""",일본 수도권 일부 지역에 신종 코로나바이러스 감염증(코로나19) 긴급사태가 발령 중...,society
3134,[슬라이드 뉴스] 이지은 누구? '금홍아 금홍아'로 신인상 휩쓴 '청춘스타',숨진 채 발견된 배우 이지은에 대해 관심이 쏠리고 있다. 1971년생인 이지은은 1...,entertain
3135,"文 “검찰 공정성 신뢰 나아지지 않아… 기소·수사권 분리, 나아가야 할 방향”","문재인 캐리커처 문재인(얼굴) 대통령은 8일 ""기소권과 수사권 분리는 앞으로도 꾸준...",politics
3136,"中, '백신여권' 도입에 ""편리한 인원왕래 요구 절실""",[서울=뉴시스] 중국 외교부가 '중국판 백신 여권'에 대해 추가적인 설명을 내놓았다...,international


In [6]:
list = ['data']
min_df = 3
max_iteration = 20
NUM_TOPICS = 8

In [8]:
df['category'].unique()

array(['international', 'economy', 'society', 'sport', 'it', 'politics',
       'entertain', 'culture'], dtype=object)

In [9]:
from konlpy.tag import *
from nltk import Text

okt = Okt()
for keyword in df['category'].unique() :
    for name in list :
        name_01 = name + ".csv"
        keyword = keyword
        
        df = pd.read_csv(name_01).dropna()
        df2 = df[df['category'] == keyword]
        # Convert to list
        data = df2.cleanBody.values.tolist()

        # user text에서 명사만 추출 
        data_morph = []
        for i in range(len(data)):
            data_morph.append(" ".join(okt.nouns(data[i])))

        vectorizer = CountVectorizer(analyzer='word',       
                                min_df=min_df,   # 특정 횟수 이상 언급된 것만 따로                      
                                max_features=50000,             
                                )

        data_vectorized = vectorizer.fit_transform(data_morph)


        NUM_TOPICS = NUM_TOPICS # 분류하는 카테고리 숫자
        lda_model = LatentDirichletAllocation(
                                n_components=NUM_TOPICS,              
                                max_iter=max_iteration,               
                                learning_method='online',   
                                random_state=100,          
                                batch_size=128,            
                                evaluate_every = -1,       
                                #n_jobs = -1,               
                            )
        lda_output = lda_model.fit_transform(data_vectorized)
        log = lda_model.score(data_vectorized)
        per = lda_model.perplexity(data_vectorized)
        
        # Create Document - Topic Matrix
        lda_output = lda_model.transform(data_vectorized)

        # column names
        topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]

        # index names
        docnames = ["Doc" + str(i) for i in range(len(data))]

        # Make the pandas dataframe
        df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

        # Get dominant topic for each document
        dominant_topic = np.argmax(df_document_topic.values, axis=1)
        df_document_topic['dominant_topic'] = dominant_topic

        # Styling
        def color_green(val):
            color = 'green' if val > .1 else 'black'
            return 'color: {col}'.format(col=color)

        def make_bold(val):
            weight = 700 if val > .1 else 400
            return 'font-weight: {weight}'.format(weight=weight)

        # Apply Style
        df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
        
        # Create Document - Topic Matrix
        lda_output = lda_model.transform(data_vectorized)

        # column names
        topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]

        # index names
        docnames = ["Doc" + str(i) for i in range(len(data))]

        # Make the pandas dataframe
        df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

        # Get dominant topic for each document
        dominant_topic = np.argmax(df_document_topic.values, axis=1)
        df_document_topic['dominant_topic'] = dominant_topic

        # Styling
        def color_green(val):
            color = 'green' if val > .1 else 'black'
            return 'color: {col}'.format(col=color)

        def make_bold(val):
            weight = 700 if val > .1 else 400
            return 'font-weight: {weight}'.format(weight=weight)

        # Apply Style
        df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
        print(df_document_topics)
        
        name_02 = keyword + "_reference" + ".html"
            
        pyLDAvis.enable_notebook()
        p = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer)
        
        pyLDAvis.save_html(p,name_02)
        
            # Show top n keywords for each topic
        def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
            keywords = np.array(vectorizer.get_feature_names())
            topic_keywords = []
            for topic_weights in lda_model.components_:
                top_keyword_locs = (-topic_weights).argsort()[:n_words]
                topic_keywords.append(keywords.take(top_keyword_locs))
            return topic_keywords

        topic_keywords = show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=11) 
        
        name_03 = keyword +  "_reference" +".csv"
        pd.DataFrame(topic_keywords).to_csv(name_03, index=False, encoding='utf-8-sig')
        print(pd.DataFrame(topic_keywords));

    pyLDAvis.enable_notebook()
    pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/usr/local/lib/python3.8/dist-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/local/lib/python3.8/dist-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/local/lib/python3.8/dist-package

    0    1    2    3    4    5    6     7    8    9    10
0  대통령  트럼프  바이든   미국  당선인   선거   대선    중국   북한  행정부   결과
1   운하   사고   호기  수에즈   작업   선박   볼더    에버  텍사스   사망  원자로
2  대통령  트럼프   탄핵   미국   의회   대한   의원    상원  민주당   이번   하원
3   화성   매출   진입  우주선   고스  라이프   차량    업종   유행   데뷔   오른
4   백신  미얀마   접종  코로나   미국   군부  화이자    시위   영국   승인   현지
5  트럼프  대통령  줄리아  백악관   파티  마스크  변호사   주니어   참석   줄리   미국
6   백신   접종  코로나   중국   일본   발생   정부    지진   지역   의료   총리
7  코로나   미국   영국   진자   현지   시간  사망자  바이러스   지난   후보   개표


/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/usr/local/lib/python3.8/dist-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/local/lib/python3.8/dist-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/usr/local/lib/python3.8/dist-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/local/lib/python3.8/dist-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/local/lib/python3.8/dist-package

    0    1     2   3     4    5   6    7      8    9    10
0   청약   주택    직원  공급    사업   부정  개발  신도시     정보   분양  국토부
1   상승   달러    하락  증가    삼성   올해  기록   전자    반도체   대비   보유
2   백신  코로나    진자  항공  아시아나   발생  지역   대한     치료   허가   한진
3   기업   시장   전기차  미국    사업  배터리  계획   기술     회장   업체   통해
4   공단   개성    정부  기업    대책   지원  지급   재개     원금   주택   마련
5  공매도  현대차    애플  자율    그룹   주행  주가   금지     종목  투자자   한국
6   배송   고객   이벤트  이번   서비스   바로  적용  코로나  크리스마스   제공   도착
7   주가   금리  비트코인  거래    국채  테슬라  투자   가치    브라질   미국   기술


...

/usr/local/lib/python3.8/dist-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/local/lib/python3.8/dist-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/usr/local/lib/python3.8/dist-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/local/lib/python3.8/dist-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/local/lib/python3.8/dist-package

    0    1    2       3    4    5   6    7    8   9     10
0  황하나   혐의   방역      마약  코로나   지난  자신   라며   관리  투약    경찰
1   대해   사실   자신      생각   사과  소속사  입장   당시   대한  통해    이후
2  박수홍   친형   법인      주장   횡령   측은  명의   대표   대한  갈등  여자친구
3  홍현희   빌딩  한남동      매입   매각   외모  건물  고소장   접수  소재    수년
4  조병규   출연   학폭      사람   폭력   의혹  주장  요아리   학교  태연    사실
5  양지은   트롯   무대      미스   출연   사랑  지난   통해   영화  자리   라운드
6   남편   방송   아내      결혼  함소원   출연  공개   사람   이혼  사랑    라며
7  콘텐츠  글로벌  카카오  엔터테인먼트  시상식  플랫폼  음악   산업  경쟁력  사업    지민


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/usr/local/lib/python3.8/dist-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/local/lib/python3.8/dist-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/local/lib/python3.8/dist-package

    0      1     2      3    4    5     6     7    8    9   10
0  미나리     후보    영화    윤여정   올해   외신  아카데미    미국   감독   연상  연기
1  윤여정     후보    미국   아카데미  미나리  시상식    영화   오스카   배우  비평가  협회
2   아이     작품    영화     생각   후보   감독    촬영    전시   행사   대표  사람
3   작품     영화    감독     아이   생각  이야기    사람    전시  캐릭터  드라마  논란
4   감독    미나리    미국  골든글로브   후보  영화상   외국어    영화   작품  시상식  한국
5   내일     영화    예정     논란   남편  시사회    배우    미래   언론   일정  배급
6   배우     영화   윤여정     후보   한국  미나리   시상식  아카데미   미국   연상  여우
7   기록  방탄소년단  문화유산     미국  빌보드   후보    유산   그래미   한국   인류  무대


In [10]:
from konlpy.tag import *
from nltk import Text

okt = Okt()
for keyword in df['category'].unique() :
    for name in list :
        name_01 = name + ".csv"
        keyword = keyword
        
        df = pd.read_csv(name_01).dropna()
        df2 = df[df['category'] == keyword]
        # Convert to list
        data = df2.cleanBody.values.tolist()

        # user text에서 명사만 추출 
        data_morph = []
        for i in range(len(data)):
            data_morph.append(" ".join(okt.nouns(data[i])))

        vectorizer = TfidfVectorizer(analyzer='word',       
                                min_df=min_df,   # 특정 횟수 이상 언급된 것만 따로                      
                                max_features=50000,             
                                )

        data_vectorized = vectorizer.fit_transform(data_morph)


        NUM_TOPICS = NUM_TOPICS # 분류하는 카테고리 숫자
        lda_model = LatentDirichletAllocation(
                                n_components=NUM_TOPICS,              
                                max_iter=max_iteration,               
                                learning_method='online',   
                                random_state=100,          
                                batch_size=128,            
                                evaluate_every = -1,       
                                #n_jobs = -1,               
                            )
        lda_output = lda_model.fit_transform(data_vectorized)
        log = lda_model.score(data_vectorized)
        per = lda_model.perplexity(data_vectorized)
        
        # Create Document - Topic Matrix
        lda_output = lda_model.transform(data_vectorized)

        # column names
        topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]

        # index names
        docnames = ["Doc" + str(i) for i in range(len(data))]

        # Make the pandas dataframe
        df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

        # Get dominant topic for each document
        dominant_topic = np.argmax(df_document_topic.values, axis=1)
        df_document_topic['dominant_topic'] = dominant_topic

        # Styling
        def color_green(val):
            color = 'green' if val > .1 else 'black'
            return 'color: {col}'.format(col=color)

        def make_bold(val):
            weight = 700 if val > .1 else 400
            return 'font-weight: {weight}'.format(weight=weight)

        # Apply Style
        df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
        
        # Create Document - Topic Matrix
        lda_output = lda_model.transform(data_vectorized)

        # column names
        topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]

        # index names
        docnames = ["Doc" + str(i) for i in range(len(data))]

        # Make the pandas dataframe
        df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

        # Get dominant topic for each document
        dominant_topic = np.argmax(df_document_topic.values, axis=1)
        df_document_topic['dominant_topic'] = dominant_topic

        # Styling
        def color_green(val):
            color = 'green' if val > .1 else 'black'
            return 'color: {col}'.format(col=color)

        def make_bold(val):
            weight = 700 if val > .1 else 400
            return 'font-weight: {weight}'.format(weight=weight)

        # Apply Style
        df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
        print(df_document_topics)
        
        name_02 = keyword + "_reference" + ".html"
            
        pyLDAvis.enable_notebook()
        p = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer)
        
        pyLDAvis.save_html(p,name_02)
        
            # Show top n keywords for each topic
        def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
            keywords = np.array(vectorizer.get_feature_names())
            topic_keywords = []
            for topic_weights in lda_model.components_:
                top_keyword_locs = (-topic_weights).argsort()[:n_words]
                topic_keywords.append(keywords.take(top_keyword_locs))
            return topic_keywords

        topic_keywords = show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=11) 
        
        name_03 = keyword +  "_reference" +".csv"
        pd.DataFrame(topic_keywords).to_csv(name_03, index=False, encoding='utf-8-sig')
        print(pd.DataFrame(topic_keywords));

    pyLDAvis.enable_notebook()
    pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/usr/local/lib/python3.8/dist-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/local/lib/python3.8/dist-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/local/lib/python3.8/dist-package

     0    1    2     3      4    5    6     7    8    9    10
0  스페이스   화성   스타    여성    우주선   착륙   이사  하시모토   폭발   비행   회장
1    열차   대만  화장실    호국     비밀   터널   린지    경호   소년   잠정   창문
2   레이디   가가   산책    마리     주인   총격   안정    지장   남자  의사당   생명
3   세이브   부금  리더십  아메리카   웹사이트   비용   기부    분배   모금   회계   검표
4    백신  미얀마   군부    시위     접종   미국   북한    중국  바이든  쿠데타   경찰
5   코로나   일본   진자   대통령     중국   접종  바이든    축구   긴급   미국   백신
6    운하  수에즈   에버    선박  컨테이너선   라비   좌초    청장   븐호   예인  이집트
7   트럼프  대통령  코로나    미국     진자  바이든   백신    탄핵  당선인   선거   연방


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/usr/local/lib/python3.8/dist-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/local/lib/python3.8/dist-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/local/lib/python3.8/dist-package

    0     1     2    3    4    5    6    7    8     9      10
0   청약    주택    공급   부정   당첨  국토부   분양   자격  아파트    단지     가구
1   수출    취업    사업   구간   감소   증가   수입   전자   인천    가장  디스플레이
2   직원    배송   신도시  이벤트   고객   정보   보상   업무   토지    개발    텔레콤
3   사고   아파트    차량   직원   운수   후원   택시  코로나   양평   거래량     종사
4   부부  신혼부부    유족  공무원  광화문   전보   연차   잔액   벌이  국정감사     자녀
5  부회장    삼성    준법  이재용   선고   재판  상속세   파기   환송    보사     농단
6  신세계    증여   부회장   총괄   내야  이마트   부연   아들   할증    부친     납세
7   공단    달러  비트코인   주가   미국   외환   거래   상승   기업    금리     하락


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/usr/local/lib/python3.8/dist-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/local/lib/python3.8/dist-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/local/lib/python3.8/dist-package

...

   0    1    2    3    4    5    6     7    8    9   10
0  수출   증대   매년   개척   겨울   요건   상담    소독   상당   조사  수당
1  후보   부부  대통령   국민  김여정   의원   이번    규명   야권   인사  공급
2  북한   미국   발사  김정은   대회  노동당   부부   미사일  바이든   한미  열병
3  후보   학습   공사   대해   의원  주차장   현장   김현미   법원  감사원  교육
4  후보   의원   국민  대통령   대표  민주당   장관  서울시장   일화   총장  정부
5  공항   의원   장관  대통령   김해  가덕도   검증    수용   재선   후보  사진
6  접종  대통령   백신   대표   국민   장관   사면   개정안   후보  민주당  의원
7  후보   차장  후보자   국민   처법   장관  외교부    조사   의원   주택  추천


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/usr/local/lib/python3.8/dist-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/local/lib/python3.8/dist-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/local/lib/python3.8/dist-package

     0      1    2      3    4    5     6    7    8    9     10
0   박수홍     친형   법인     사랑   횡령   관심    갈등   명의   주장   통장  여자친구
1    남편    함소원  조병규     자신   결혼   이혼    아내   출연   소식   방송    배우
2    신인     러브   모델     발견   영화   협회    경찰  드라마  젊은이  며느리    원인
3    지민  뮤직비디오   안무  방탄소년단   레이   착용    멤버   댄스   공개   영상    버전
4    트롯     미스  조병규    양지은   태연   폭력    학교  시상식  열애설   사람   진달래
5   박시연    설민석  이승환   음주운전   음원   대리  전국체전   운전  세계사  윤석열    사고
6   요아리     게인   싱어     빅죠  생방송   오늘    자퇴  박나래  이나래   독자    의혹
7  아나운서    콘텐츠  글로벌    카카오  황하나  김보경   아이언   마약  경쟁력   혐의   플랫폼


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/usr/local/lib/python3.8/dist-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/local/lib/python3.8/dist-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/local/lib/python3.8/dist-package

        0     1     2     3    4       5     6      7    8     9      10
0     문화유산    인류    유산    행사  위원회      문화    국가     대표   보존  우리나라     정부
1       감독   미나리   윤여정    후보   미국      영화  아카데미     배우  시상식   오스카  골든글로브
2       전시    지원    생각    아트   작품     아버지    예술     사람  제작비    배경     계획
3    방탄소년단   빌보드    아이   그래미   감독  다이너마이트   캐릭터    드라마   영화    기록     무대
4       강조  독립영화   이야기  할리우드   전국      작업    시작     기록   조성    진행     전시
5  미국배우조합상    평단  오스카상    매년   중인      시대    비롯     미래   이유   시사회     조성
6      윤여정    영화    배우    한국  시상식      사람   미나리    한예리   작품    여우   아카데미
7       작품    대상    남녀   라미란   후보      효과    배우  애니메이션   주연    최고     세계
